In [1]:
import cv2 as cv
import numpy as np
import os

In [3]:
%run ./constants.ipynb

In [36]:
class ImageProcessor:
    """
    A class for processing images, color space conversion, resizing, and cropping.
    
    Attributes:
        images_dir: The directory containing the images.
        image: The currently loaded image.
    """
    def __init__(
            self, 
            images_dir: os.DirEntry, 
            image_dimensions: tuple | None = None,
            crop_dimensions: list | None = None,
            save_folder: str | None = None,
        ) -> None:
        """
        Initialize the ImageProcessor with the directory containing the images.
        
        Args:
            images_dir: The directory containing the images.
        """
        self.images_dir: os.DirEntry = images_dir
        self.image_paths = [os.path.join(images_dir, f) for f in os.listdir(images_dir) if f.endswith('.jpg')]
        self.image_dimensions: tuple = image_dimensions
        self.crop_dimensions: list[int] = crop_dimensions
        self.save_folder: os.DirEntry = save_folder or "preprocessed"
        self.image: np.array = None
        
    
    def process_images(self):
        """
        Process all valid images in the directory using the specified processing method.
        """
        for image_path in self.image_paths:
            self._load_image(image_path)
            self._bgr_to_rgb()
            # @NOTE: crop before image resizing to correct for aspect
            if self.crop_dimensions:
                self._crop_image(*self.crop_dimensions)
            if self.image_dimensions:
                self._resize_images(self.image_dimensions)
            self._write(image_path)
            
    
    def _load_image(self, image_path: str) -> None:
        """
        Load an image from the specified path.
        
        Args:
            image_path: The path to the image file.
        """
        self.image = cv.imread(os.path.join(self.images_dir, image_path))
        if self.image is None:
            raise ValueError("No image loaded.")

    
    def _bgr_to_rgb(self) -> None:
        """
        Convert the loaded image from BGR to RGB color space.
        """
        self.image = cv.cvtColor(self.image, cv.COLOR_BGR2RGB)
    
    def _resize_images(self, target_size) -> None:
        """
        Resize all images in the directory to the specified target size and save them in a subfolder.
        
        Args:
            target_size: The target for the resized images.
        """
        # Aspect ratio of the original image
        original_height, original_width = self.image.shape[:2]
        aspect_ratio = original_width / original_height
        
        # Adjust target to maintain aspect ratio
        target_width, target_height = target_size
        if target_width / target_height > aspect_ratio:
            target_width = int(target_height * aspect_ratio)
        else:
            target_height = int(target_width / aspect_ratio)
        
        self.image = cv.resize(self.image, (target_width, target_height))
    
    def _crop_image(self, left: int, right: int, top: int, bottom: int) -> None:
        """
        Crop the currently loaded image.
        
        Args:
            left: Number of pixels to extract from the left side of the image.
            right: Number of pixels to extract from the right side of the image.
            top: Number of pixels to extract from the top side of the image.
            bottom: Number of pixels to extract from the bottom side of the image.
        """
        height, width = self.image.shape[:2]
        self.image = self.image[top:height-bottom, left:width-right]
    
    def _write(self, image_path):
        filename = os.path.basename(image_path)
        save_path = os.path.join(self.images_dir, self.save_folder)
        file_path = os.path.join(save_path, filename)
        os.makedirs(save_path, exist_ok=True)
        cv.imwrite(file_path, cv.cvtColor(self.image, cv.COLOR_RGB2BGR))

image_processor = ImageProcessor(IMAGE_DIR, crop_dimensions=[0,0,0,70], image_dimensions=(250,250))
image_processor.process_images()